In [1]:
import pandas as pd

IMPORT DATA:

In [6]:
branded_food_df = pd.read_csv("foodData/branded_food.csv")
branded_food_df.head()

C:\Users\eli_m\AppData\Local\Temp\ipykernel_20508\2971101322.py:1: DtypeWarning: Columns (2,3,4,6,9,12,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  branded_food_df = pd.read_csv("foodData/branded_food.csv")


,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,household_serving_fulltext,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
0,1105904,Richardson Oilseed Products (US) Limited,NaN,NaN,27000612323,Vegetable Oil,NaN,15.0,ml,NaN,Oils Edible,GDSN,NaN,2020-10-02,2020-11-13,United States,NaN,NaN,NaN,NaN
1,1105905,CAMPBELL SOUP COMPANY,NaN,NaN,51000198808,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,240.0,ml,NaN,Herbs/Spices/Extracts,GDSN,NaN,2020-09-12,2020-11-13,United States,NaN,NaN,NaN,NaN
2,1105906,CAMPBELL SOUP COMPANY,NaN,NaN,51000213273,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,440.0,g,NaN,Prepared Soups,GDSN,NaN,2020-09-01,2020-11-13,United States,NaN,NaN,NaN,NaN
3,1105907,CAMPBELL SOUP COMPANY,NaN,NaN,51000213303,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,440.0,g,NaN,Prepared Soups,GDSN,NaN,2020-09-01,2020-11-13,United States,NaN,NaN,NaN,NaN
4,1105908,CAMPBELL SOUP COMPANY,NaN,NaN,51000224637,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,240.0,ml,NaN,Herbs/Spices/Extracts,GDSN,NaN,2020-10-03,2020-11-13,United States,NaN,NaN,NaN,NaN


HELPER FUNCTIONS FOR CLEANING DF:

In [7]:
# Define a function to count non-zero digits in the beginning characters beyond the first 12 digits
def count_non_zero_digits(upc_code):
    # Determine the length of the substring to check based on the total length of the UPC code
    extra_length = len(upc_code) - 12
    if extra_length > 0:
        # Select the substring of beginning characters beyond the first 12 digits
        beginning_characters = upc_code[0:extra_length]
        # Count the occurrences of non-zero digits in the beginning characters
        non_zero_count = sum(1 for digit in beginning_characters if digit != '0')
        return non_zero_count
    else:
        return 0
    
# Function to clean the UPC codes to all be of length 12
def clean_upc(upc_code):
    # Remove leading zeros
    cleaned_upc = upc_code
    
    # If the length of the cleaned UPC code is greater than 12, remove excess digits from the front
    if len(cleaned_upc) > 12:
        cleaned_upc = cleaned_upc[-12:]
    
    # If the length of the cleaned UPC code is less than 12, add leading zeros
    if len(cleaned_upc) < 12:
        cleaned_upc = '0' * (12 - len(cleaned_upc)) + cleaned_upc
    
    return cleaned_upc


In [8]:
#deal with terms like dairy-free and non-dairy

def combine_dairy_free(words):
    combined_words = []
    i = 0
    while i < len(words):
        if i < len(words) - 1 and words[i] in ['DAIRY'] and words[i+1] == 'FREE':
            combined_words.append(words[i] + 'FREE')
            i += 2
        elif i < len(words) - 1 and words[i] in ['NON'] and words[i+1] == 'DAIRY':
            combined_words.append(words[i] + 'DAIRY')
            i += 2
        else:
            combined_words.append(words[i])
            i += 1
    return combined_words

#function to handle nut milk in ingredient list
def combine_nut_milk(words):
    combined_words = []
    i = 0
    while i < len(words):
        if i < len(words) - 1 and words[i] in ['ALMOND', 'WALNUT', 'CASHEW', 'COCONUT', 'RICE', 'OAT', 'PEA', 'HAZELNUT', 'QUINOA', 'HEMP', 'PLANT', 'FLAXSEED', 'BANANA', 'PISTACHIO'] and words[i+1] == 'MILK':
            combined_words.append(words[i] + 'MILK')
            i += 2  # Skip the next word ('milk')
        else:
            combined_words.append(words[i])
            i += 1
    return combined_words


In [9]:
# Helper Function to merge information
def merge_info(group):
    merged_entry = {}
    for col in group.columns:
        if col == 'gtin_upc':
            merged_entry[col] = group[col].iloc[0]  # Keep the UPC code
        # elif col == 'ingredients':
        #     unique_ingredients = set()
        #     for ingredients_list in group[col]:
        #         unique_ingredients.update(ingredients_list)  # Add unique ingredients from each entry
        #     merged_entry[col] = list(unique_ingredients)  # Convert unique ingredients to list

        else:
             merged_entry[col] = group[col].dropna().iloc[0] if not group[col].dropna().empty else None  # Choose non-null value or None if all are null
    return pd.Series(merged_entry)



In [ ]:
#more helper functions for strings:
import re

def remove_chars(s):
    # Define a regular expression pattern to match any of the ugly characters
    pattern = r'[\[\]{}:;.&*]'
    # Use re.sub() to replace all occurrences of the pattern with an empty string
    return re.sub(pattern, '', s)

def remove_chars2(s):
    # Define a regular expression pattern to match any of the ugly characters
    pattern = r'[\[\]{}:;.&*(),]'
    # Use re.sub() to replace all occurrences of the pattern with an empty string
    return re.sub(pattern, '', s)

def remove_chars3(s):
    # Define a regular expression pattern to match any of the ugly characters
    pattern = r'-'
    # Use re.sub() to replace all occurrences of the pattern with an empty string
    return re.sub(pattern, '', s)

def add_space_after_comma(s):
    # Define a regular expression pattern to match any comma not followed by a space
    pattern = r',(?![\s])'
    # Use re.sub() to replace all occurrences of the pattern with a comma followed by a space
    return re.sub(pattern, ', ', s)

CLEANING FUNCTION:

In [10]:
#CLEANING function
def clean_df(df):
    df.dropna(subset=['ingredients', 'branded_food_category', 'gtin_upc'], inplace=True)
    df['ingredients'] = df['ingredients'].astype('string')

    df['ingredients'] = df['ingredients'].apply(remove_chars)

    # Add spaces after commas not followed by a space
    df['ingredients'] = df['ingredients'].apply(add_space_after_comma)

    pattern = r',|\(|\)'
    # Split the ingredients column based on the pattern
    df['ingredients'] = df['ingredients'].str.split(' ')

    # Convert ingredients to uppercase
    df['ingredients'] = df['ingredients'].apply(lambda x: [ingredient.upper() for ingredient in x])

    df['ingredients'] = df['ingredients'].apply(combine_nut_milk)
    df['ingredients'] = df['ingredients'].apply(combine_dairy_free)

    df['ingredients'] = df['ingredients'].apply(lambda x: [remove_chars2(ing_list) for ing_list in x])
    

    # Make upc codes of type string
    df['gtin_upc'] = df['gtin_upc'].astype('string')

    df['gtin_upc'] = df['gtin_upc'].apply(remove_chars3)

    # SECTION THAT ELIMINATES UPC WITH LEADING DIGITS THAT ARENT ZERO (LOSING DATA)
    mask = df['gtin_upc'].apply(count_non_zero_digits) == 0
    filtered_df = df[mask]

    # Apply the function to each UPC code in the 'upc_code' column
    filtered_df['gtin_upc'] = filtered_df['gtin_upc'].apply(clean_upc)

    grouped_scan = filtered_df.groupby('gtin_upc')
    merged_df = grouped_scan.apply(merge_info).reset_index(drop=True)

    return merged_df


In [11]:
#running function:
cleaned_df = clean_df(branded_food_df)
cleaned_df.head()

C:\Users\eli_m\AppData\Local\Temp\ipykernel_20508\863748443.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['gtin_upc'] = filtered_df['gtin_upc'].apply(clean_upc)


,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,household_serving_fulltext,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
0,1082179,KOOL-AID SINGLES,None,None,0000 4338107,"[SUGAR, FRUCTOSE, CITRIC, ACID, CONTAINS, LESS...",None,16.0,g,1 Packet,Powdered Drinks,LI,None,2020-06-26,2020-08-27,United States,None,None,None,None
1,555505,AHOLD,None,None,000000000000,"[ORGANIC, BLUE, CORN, ORGANIC, SUNFLOWER, OIL,...",None,28.0,g,1 ONZ,"Chips, Pretzels & Snacks",LI,None,2018-04-16,2019-04-01,United States,None,None,None,None
2,359586,TRADER JOE'S,TRADER JOE'S,None,000000005487,"[UNPEELED, POTATOES, SUNFLOWER, OIL, SEA, SALT]",None,28.0,g,1 ONZ,"Chips, Pretzels & Snacks",LI,198 g/7 oz,2018-08-12,2019-04-01,United States,None,None,None,None
3,557566,TRADER JOE'S,TRADER JOE'S,None,000000006163,"[GHERKINS, CUCUMBERS, WATER, VINEGAR, SALT, ON...",None,28.0,g,1 ONZ,"Pickles, Olives, Peppers & Relishes",LI,350 g,2018-01-20,2019-04-01,United States,None,None,None,None
4,386120,TRADER JOE'S,None,None,000000010894,"[ORGANIC, TOMATOES, ORGANIC, TOMATO, JUICE, FR...",None,130.0,g,0.5 cup,Tomatoes,LI,None,2017-06-25,2019-04-01,United States,None,None,None,None


CHECKING THAT DATA IS CORRECT:

In [12]:
len(cleaned_df)

425545

In [13]:
upc_code_lengths = cleaned_df['gtin_upc'].str.len()

# Count occurrences of each length of UPC codes
length_counts = upc_code_lengths.value_counts()

# Print the histogram format
print("Histogram of UPC code lengths:")
print(length_counts)

Histogram of UPC code lengths:
12    425545
Name: gtin_upc, dtype: int64


RECOMMENDING BASED ON UPC:

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Helper Function to merge information
def merge_info(group):
    merged_entry = {}
    for col in group.columns:
        if col == 'gtin_upc':
            merged_entry[col] = group[col].iloc[0]  # Keep the UPC code
        else:
             merged_entry[col] = group[col].dropna().iloc[0] if not group[col].dropna().empty else None  # Choose non-null value or None if all are null
    return pd.Series(merged_entry)

def add_plural(input_strings):
    plural_strings = [string + '|' + string + 's' for string in input_strings]
    return '|'.join(plural_strings)


#helper function to rank recommendations
def rank_products(allergen_df, no_allergen_df):
    # Flatten the list of ingredients into a single string for each product
    allergen_ingredients = [''.join(ingredients) for ingredients in allergen_df['ingredients']]
    no_allergen_ingredients = [''.join(ingredients) for ingredients in no_allergen_df['ingredients']]

    # Combine the ingredient lists from both dataframes
    all_ingredients = allergen_ingredients + no_allergen_ingredients
    print(all_ingredients)

    # if not all_ingredients:
    #     return pd.DataFrame()

    # Convert ingredient lists into vectors
    vectorizer = CountVectorizer()
    ingredient_vectors = vectorizer.fit_transform(all_ingredients)

    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(ingredient_vectors)

    # Get similarity scores for all products except the first len(peanut_ingredients) ones
    similarity_scores = cosine_similarities[:len(allergen_ingredients), len(allergen_ingredients):]

    # Create a new column in the merged DataFrame to store the similarity scores
    no_allergen_df['similarity_score'] = similarity_scores.flatten()

    # Rank products based on similarity scores
    ranked_indices = similarity_scores.argsort(axis=1)[:, ::-1]
    ranked_products = no_allergen_df.iloc[ranked_indices.ravel()]

    return(ranked_products)

In [13]:
#upc code should be numeric input, allergen should be string
def recommender(upc_code, allergens, df):
    #single entry df with allergen product 
    scanned_df = df.loc[df['gtin_upc'] == upc_code].reset_index(drop=True)
    # if the upc is not found, return an empty df
    if scanned_df.empty:
        return scanned_df
    #grouped_scan = scanned_df.groupby('gtin_upc')
    #merged_scanned_df = grouped_scan.apply(merge_info).reset_index(drop=True)

    # Remove 's' from each term in the allergens list
    allergens = [allergen.lower().rstrip('s') for allergen in allergens]

    if ('tree nut' in allergens):
        allergens = allergens + ['almond', 'cashew', 'chestnut', 'hazelnut', 'macadamia', 'pecan', 'pistachio', 'walnut']

    if ('dairy' in allergens):
        allergens = allergens + ['milk', 'mlk', 'lactose', 'whey', 'casein', 'caseinate', 'butter', 'buttermilk', 'cheese', 'cream', 'curd', 'ghee', 'half-and-half', 'yogurt', 'skim-milk', 'milkfat', 'milkvitamin']
        

    #df without allergen as an ingredient
    allergen_string = add_plural(allergens).upper()
    allergen_words = allergen_string.split('|')
    print(allergen_words)
    without_allergen_df = df[~df['ingredients'].apply(lambda x: any(word in x for word in allergen_words))]


    product_category = scanned_df.loc[0]['branded_food_category']
    without_allergen_df = without_allergen_df[without_allergen_df['branded_food_category'] == product_category]
    #grouped_rec = without_allergen_df.groupby('gtin_upc')
    #merged_rec_df = grouped_rec.apply(merge_info).reset_index(drop=True)

    ranked_df = rank_products(scanned_df, without_allergen_df)
 
    return(ranked_df)

Testing a different upc for recommendation:

In [17]:
recommendations_tate_df = recommender('810291001064', ['dairy', 'tree nuts', 'egg', 'peanut'], ess_data_df)
recommendations_tate_df.head()

['DAIRY', 'DAIRYS', 'TREE NUT', 'TREE NUTS', 'EGG', 'EGGS', 'PEANUT', 'PEANUTS', 'ALMOND', 'ALMONDS', 'CASHEW', 'CASHEWS', 'CHESTNUT', 'CHESTNUTS', 'HAZELNUT', 'HAZELNUTS', 'MACADAMIA', 'MACADAMIAS', 'PECAN', 'PECANS', 'PISTACHIO', 'PISTACHIOS', 'WALNUT', 'WALNUTS', 'MILK', 'MILKS', 'MLK', 'MLKS', 'LACTOSE', 'LACTOSES', 'WHEY', 'WHEYS', 'CASEIN', 'CASEINS', 'CASEINATE', 'CASEINATES', 'BUTTER', 'BUTTERS', 'BUTTERMILK', 'BUTTERMILKS', 'CHEESE', 'CHEESES', 'CREAM', 'CREAMS', 'CURD', 'CURDS', 'GHEE', 'GHEES', 'HALF-AND-HALF', 'HALF-AND-HALFS', 'YOGURT', 'YOGURTS', 'SKIM-MILK', 'SKIM-MILKS', 'MILKFAT', 'MILKFATS', 'MILKVITAMIN', 'MILKVITAMINS']
["['SEMI-SWEET', 'CHOCOLATE', 'CHIPS', 'SUGAR', 'CHOCOLATE', 'LIQUOR', 'COCOA', 'BUTTER', 'MILK', 'FAT', 'SOY', 'LECITHIN', 'AN', 'EMULSIFIER', 'VANILLA', 'NATURAL', 'FLAVOR', 'BUTTER', 'UNBLEACHED', 'FLOUR', 'WHEAT', 'FLOUR', 'NIACIN', 'REDUCED', 'IRON', 'THIAMINE', 'MONONITRATE', 'RIBOFLAVIN', 'FOLIC', 'ACID', 'MALTED', 'BARLEY', 'FLOUR', 'CANE', '

,gtin_upc,ingredients,branded_food_category,similarity_score
158101,046567031996,"['UNBLEACHED', 'ENRICHED', 'FLOUR', 'WHEAT', '...",Cookies & Biscuits,0.690654
79762,030684035043,"['ENRICHED', 'WHEAT', 'FLOUR', 'WHEAT', 'FLOUR...",Cookies & Biscuits,0.686904
283347,099482508272,"['UNBLEACHED', 'ENRICHED', 'WHEAT', 'FLOUR', '...",Cookies & Biscuits,0.681667
121,000000438711,"['WHOLE', 'WHEAT', 'FLOUR', 'UNBLEACHED', 'ENR...",Cookies & Biscuits,0.652039
2000,000011784388,"['WHOLE', 'WHEAT', 'FLOUR', 'UNBLEACHED', 'ENR...",Cookies & Biscuits,0.652039


In [ ]:
recommendations_df = recommender('005150241820', ['peanut'], cleaned_df)
recommendations_df.head()

In [ ]:
string_list = recommendations_df['gtin_upc'].tolist()
print(string_list)

In [ ]:
recommendations2_df = recommender('044000002107', ['peanut', 'flour'], cleaned_df)
recommendations2_df.head()

In [ ]:
recommendations3_df = recommender('037600106252', ['peanut'], cleaned_df)
recommendations3_df.head()

In [ ]:
#almond milk example
recommendations4_df = recommender('041570052600', ['almond'], cleaned_df)
recommendations4_df.head()

In [ ]:
#almond milk example
recommendationsa_df = recommender('041570052600', ['dairy', 'almond'], cleaned_df)
recommendationsa_df.head()

In [ ]:
#almond milk example
recommendations5_df = recommender('041570052600', ['dairy'], cleaned_df)
recommendations5_df.head()

In [ ]:
#fairlife milk example
recommendations5_df = recommender('811620020039', ['dairy'], cleaned_df)
recommendations5_df.head()

In [ ]:
string_list4 = recommendations4_df['gtin_upc'].tolist()
print(string_list4[0:5])

HEATMAP OF PB REPLACEMENTS AND ALMOND JOY (showing differences between recommendations)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df1 and df2 are your pre-existing DataFrames containing the UPC codes
# Extract the UPC codes from each DataFrame

# Concatenate the two DataFrames along the rows
combined_product_df = pd.concat([recommendations_df.head(), recommendations4_df.head()], ignore_index=True)

# Flatten the list of ingredients into a single string for each product
product_ingredients = [' '.join(ingredients) for ingredients in combined_product_df['ingredients']]

# Convert ingredient lists into vectors
vectorizer = CountVectorizer()
ingredient_vectors = vectorizer.fit_transform(product_ingredients)

# Calculate cosine similarity
cosine_similarities = cosine_similarity(ingredient_vectors)

# Define custom labels for columns and row indices
custom_labels = ['Almond Butter w/ Honey', 'Chocolate/Hazelnut Spread', 'Chocolate Walnut Butter', 'Walmart Honey Almond Butter', 'Chocolate/Almond Spread', 
                 'Vanilla Quinoa Milk', 'Unsweetened Quinoa Milk', 'Macadamia Milk', 'Agave-Vanilla Quinoa Milk', 'Coconut Milk']


# Create a DataFrame for the similarity scores
similarity_df = pd.DataFrame(cosine_similarities, columns=custom_labels, index=custom_labels)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_df, annot=True, cmap="YlGnBu")
plt.title("Similarity Scores Heatmap")
plt.xlabel("Peanut Butter Replacements + Almond Milk Replacements")
plt.show()

